In [0]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 3.1MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 3.4MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae 
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

In [0]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/dw_matrix_car


In [0]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

##Feature Engineering

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue
  
  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [0]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  x = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, x, y, cv = 3, scoring = 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

##Decision Tree

In [0]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19747.81093847179, 75.95212671816309)

## Random Forest

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model(model, cat_feats)

(-18864.27843495869, 38.24233578175403)

##XGBoost

In [0]:
xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0
}

run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[17:54:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:54:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:55:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13034.249470063352, 103.73566521784606)

In [0]:
m = xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.1, seed=0)
m.fit(x,y)

imp = PermutationImportance(m, random_state=0).fit(x,y)
eli5.show_weights(imp, feature_names=cat_feats)

[17:56:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1274 ± 0.0017,param_napęd__cat
0.1159 ± 0.0025,param_stan__cat
0.1131 ± 0.0037,param_rok-produkcji__cat
0.0565 ± 0.0010,param_faktura-vat__cat
0.0482 ± 0.0014,param_moc__cat
0.0416 ± 0.0017,param_skrzynia-biegów__cat
0.0252 ± 0.0010,param_marka-pojazdu__cat
0.0247 ± 0.0014,feature_kamera-cofania__cat
0.0232 ± 0.0008,param_typ__cat
0.0198 ± 0.0006,param_pojemność-skokowa__cat


In [0]:
feats = ['param_napęd__cat',	'param_stan__cat',	'param_rok-produkcji__cat',	'param_faktura-vat__cat',	'param_moc__cat',	'param_skrzynia-biegów__cat',	'param_marka-pojazdu__cat',	'feature_kamera-cofania__cat',	'param_typ__cat',	'param_pojemność-skokowa__cat',	'seller_name__cat',	'param_wersja__cat',	'feature_wspomaganie-kierownicy__cat',	'param_model-pojazdu__cat',	'feature_system-start-stop__cat',	'param_kod-silnika__cat',	'feature_asystent-pasa-ruchu__cat',	'feature_łopatki-zmiany-biegów__cat',	'feature_światła-led__cat',	'feature_czujniki-parkowania-przednie__cat']
run_model(xgb.XGBRegressor(**xgb_params), feats)


[18:14:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:14:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:15:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13407.229151187508, 47.968383519686824)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x :-1 if str(x) == 'None' else int(x))

feats = ['param_napęd__cat',	'param_stan__cat',	'param_rok-produkcji',	'param_faktura-vat__cat',	'param_moc__cat',	'param_skrzynia-biegów__cat',	'param_marka-pojazdu__cat',	'feature_kamera-cofania__cat',	'param_typ__cat',	'param_pojemność-skokowa__cat',	'seller_name__cat',	'param_wersja__cat',	'feature_wspomaganie-kierownicy__cat',	'param_model-pojazdu__cat',	'feature_system-start-stop__cat',	'param_kod-silnika__cat',	'feature_asystent-pasa-ruchu__cat',	'feature_łopatki-zmiany-biegów__cat',	'feature_światła-led__cat',	'feature_czujniki-parkowania-przednie__cat']
run_model(xgb.XGBRegressor(**xgb_params), feats)

[18:18:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:18:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:18:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11364.016578827623, 80.05911838700821)

In [0]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]) )
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x :-1 if str(x) == 'None' else int(x))

feats = ['param_napęd__cat',	'param_stan__cat',	'param_rok-produkcji',	'param_faktura-vat__cat',	'param_moc',	'param_skrzynia-biegów__cat',	'param_marka-pojazdu__cat',	'feature_kamera-cofania__cat',	'param_typ__cat',	'param_pojemność-skokowa__cat',	'seller_name__cat',	'param_wersja__cat',	'feature_wspomaganie-kierownicy__cat',	'param_model-pojazdu__cat',	'feature_system-start-stop__cat',	'param_kod-silnika__cat',	'feature_asystent-pasa-ruchu__cat',	'feature_łopatki-zmiany-biegów__cat',	'feature_światła-led__cat',	'feature_czujniki-parkowania-przednie__cat']
run_model(xgb.XGBRegressor(**xgb_params), feats)

[19:56:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:56:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:56:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9724.786775066808, 84.17027602984261)

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ','')) )

feats = ['param_napęd__cat',	'param_stan__cat',	'param_rok-produkcji',	'param_faktura-vat__cat',	'param_moc__cat',	'param_skrzynia-biegów__cat',	'param_marka-pojazdu__cat',	'feature_kamera-cofania__cat',	'param_typ__cat',	'param_pojemność-skokowa',	'seller_name__cat',	'param_wersja__cat',	'feature_wspomaganie-kierownicy__cat',	'param_model-pojazdu__cat',	'feature_system-start-stop__cat',	'param_kod-silnika__cat',	'feature_asystent-pasa-ruchu__cat',	'feature_łopatki-zmiany-biegów__cat',	'feature_światła-led__cat',	'feature_czujniki-parkowania-przednie__cat']
run_model(xgb.XGBRegressor(**xgb_params), feats)



[20:03:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:03:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:04:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10408.932533939063, 101.32233224391379)